In [5]:
import os
from dotenv import find_dotenv, load_dotenv
from rich import print as rprint
from tqdm import tqdm

load_dotenv(find_dotenv())


%load_ext rich

In [2]:
import cohere
from cohere import ClassifyExample

co = cohere.Client()

# Documentation

## Classify

In [10]:
examples = [
    ClassifyExample(text="Dermatologists don't like her!", label="Spam"),
    ClassifyExample(text="'Hello, open to this?'", label="Spam"),
    ClassifyExample(text="I need help please wire me $1000 right now", label="Spam"),
    ClassifyExample(text="Nice to know you ;)", label="Spam"),
    ClassifyExample(text="Please help me?", label="Spam"),
    ClassifyExample(text="Your parcel will be delivered today", label="Not spam"),
    ClassifyExample(
        text="Review changes to our Terms and Conditions", label="Not spam"
    ),
    ClassifyExample(text="Weekly sync notes", label="Not spam"),
    ClassifyExample(text="'Re: Follow up from today's meeting'", label="Not spam"),
    ClassifyExample(text="Pre-read for tomorrow", label="Not spam"),
]

inputs = [
    "Confirm your email address",
    "hey i need u to send some $",
]

response = co.classify(
    inputs=inputs,
    examples=examples,
)
print(response)


ClassifyResponse(
    id='546058c3-50cf-41c2-acbc-4431a2c2bad2',
    classifications=[
        ClassifyResponseClassificationsItem(
            id='3af394b5-74ae-4971-8352-004cd1d6852e',
            input='Confirm your email address',
            prediction='Not spam',
            predictions=['Not spam'],
            confidence=0.5661598,
            confidences=[0.5661598],
            labels={
                'Not spam': ClassifyResponseClassificationsItemLabelsValue(confidence=0.5661598),
                'Spam': ClassifyResponseClassificationsItemLabelsValue(confidence=0.43384025)
            },
            classification_type='single-label'
        ),
        ClassifyResponseClassificationsItem(
            id='99711366-9851-4ea2-9734-32a1fcdb657a',
            input='hey i need u to send some $',
            prediction='Spam',
            predictions=['Spam'],
            confidence=0.9909811,
            confidences=[0.9909811],
            labels={
                'Not spam': ClassifyResponseClassificationsItemLabelsValue(confidence=0.009018883),
                'Spam': ClassifyResponseClassificationsItemLabelsValue(confidence=0.9909811)
            },
            classification_type='single-label'
        )
    ],
    meta=ApiMeta(
        api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None),
        billed_units=ApiMetaBilledUnits(
            input_tokens=None,
            output_tokens=None,
            search_units=None,
            classifications=2
        ),
        tokens=None,
        warnings=None
    )
)

## Define the documents

In [8]:
documents = [
    {
        "title": "The quick brown fox",
        "text": "The quick brown fox jumps over the lazy dog",
        "author": "Aesop",
    },
    {
        "title": "Cats are cute",
        "text": "Cats are cute and fluffy",
        "author": "Aesop",
    },
    {
        "title": "Tall penguins",
        "text": "Emperor penguins are the tallest.",
        "author": "Vlad",
    },
    {
        "title": "Penguin habitats",
        "text": "Emperor penguins only live in Antarctica.",
        "author": "Vlad",
    },
    {
        "title": "Penguin diet",
        "text": "Emperor penguins eat fish.",
    }
]

In [11]:
response = co.chat_stream(
    preamble="Only return in Markdown format.",
    message="What books did Vlad write?",
    documents=documents,
)


citations = []
cited_documents = []

for event in response:
    if event.event_type == "text-generation":
        print(event.text, end="")

    elif event.event_type == "citation-generation":
        citations.extend(event.citations)

    elif event.event_type == "stream-end":
        cited_documents.extend(event.response.documents)


if citations:
    print("\n\nCitations:")
    for citation in citations:
        print(citation)

if cited_documents:
    print("\n\nCited Documents:")
    for document in cited_documents:
        print(document)


Vlad wrote the following books:
- *Tall Penguins*
- *Penguin Habitats*.

Citations:
start=35 end=48 text='Tall Penguins' document_ids=['doc_2']
start=53 end=69 text='Penguin Habitats' document_ids=['doc_3']


Cited Documents:
{'author': 'Vlad', 'id': 'doc_2', 'text': 'Emperor penguins are the tallest.', 'title': 'Tall penguins'}
{'author': 'Vlad', 'id': 'doc_3', 'text': 'Emperor penguins only live in Antarctica.', 'title': 'Penguin habitats'}
